In [ ]:
import time
import sys
import inspect #디버깅용
import torch
# import tensorflow as tf
import torchvision
# import ninja
import click
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
import numpy as np
import pickle
import cv2
%matplotlib inline
from IPython.display import clear_output

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

def time_stamp():
    temp = r"%Y-%m-%d %H:%M:%S"
    current_frame = inspect.currentframe()
    tmp = f"{time.strftime(temp, time.localtime(time.time()))}: [{inspect.getframeinfo(current_frame).filename}]:"
    return tmp

def version_all():
    print(f"{time_stamp()}Python 버전📌 {sys.version_info}")
    print(f"{time_stamp()}Pytorch 버전📌 {torch.__version__}")
    print(f"{time_stamp()}torchvision 버전📌 {torchvision.__version__}")
    # print(f"{time_stamp()}ninja 버전📌 {ninja.__version__}")
    # print(f"{time_stamp()}Tensorflow 버전📌 {tf.__version__}")
    print(f"{time_stamp()}Torch GPU 가능여부📌 {torch.torch.cuda.is_available()}")
    # print(f"{time_stamp()}Tensorflow GPU 가능여부📌 {tf.config.list_physical_devices('GPU')}") # 텐서플로우 2.x버전
    # print(f"{time_stamp()}Tensorflow GPU 가능여부📌 {device_lib.list_local_devices()}") # 텐서플로우 1.x버전
    print(f"{time_stamp()}Click Version📌 {click.__version__}") # 텐서플로우 1.x버전
    # print(f"{time_stamp()}cudatoolkit Version📌 {cudatoolkit.__version__}")
    print(f"{time_stamp()}Numpy Version📌 {np.__version__}")
    
# GPU 사용 설정
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# if len(physical_devices) > 0:
    # tf.config.experimental.set_memory_growth(physical_devices[0], True)
    # print("GPU 설정 완료")
# else:
    # print("GPU를 찾을 수 없습니다. CPU")


version_all()
!nvcc -V